In [1]:
from predict import *


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 47, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 64)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 23, 64)           256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 23, 64)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 128)           16512     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 128)          0

In [2]:
## https://www.codepile.net/pile/ey9KAnxn
import sys
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import cv2

face_cascade = cv2.CascadeClassifier('assets/haarcascade_frontalface_default.xml')

class MainWindow(QWidget):
    def __init__(self):
        super(MainWindow, self).__init__()

        self.VBL = QVBoxLayout()

        self.FeedLabel = QLabel()
        self.VBL.addWidget(self.FeedLabel)

        self.CancelBTN = QPushButton("Cancel")
        self.CancelBTN.clicked.connect(self.CancelFeed)
        self.VBL.addWidget(self.CancelBTN)


        self.Worker1 = Worker1()

        self.Worker1.start()
        self.Worker1.ImageUpdate.connect(self.ImageUpdateSlot)
        
        self.textOutput = QTextEdit() 
        self.VBL.addWidget(self.textOutput)
        
        self.setLayout(self.VBL)

    def ImageUpdateSlot(self, Image):
        self.FeedLabel.setPixmap(QPixmap.fromImage(Image))

    def CancelFeed(self):
        self.Worker1.stop()

class Worker1(QThread):
    ImageUpdate = pyqtSignal(QImage)
    def run(self):
        self.ThreadActive = True
        Capture = cv2.VideoCapture(0)
        i = 0
        while self.ThreadActive:
            ret, frame = Capture.read()
            if ret:
                Image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x,y,w,h) in faces:
                    cv2.rectangle(Image,(x,y),(x+w,y+h),(255,0,0),2)
                    face_gray = gray[y:y+h, x:x+w]
                FlippedImage = cv2.flip(Image, 1)
                if i == 30:
                    print(face_gray.shape)
                    print(get_prediction(face_gray))
                    i= 0
                else:
                    i+=1
                ConvertToQtFormat = QImage(FlippedImage.data, FlippedImage.shape[1], FlippedImage.shape[0], QImage.Format_RGB888)
                Pic = ConvertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
                self.ImageUpdate.emit(Pic)
    def stop(self):
        self.ThreadActive = False
        self.quit()

if __name__ == "__main__":
    App = QApplication(sys.argv)
    Root = MainWindow()
    Root.show()
    sys.exit(App.exec())

UnboundLocalError: local variable 'face_gray' referenced before assignment

SystemExit: 0

/home/oobonioo/miniconda3/envs/happy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
